In [1]:
import torch
import scipy.signal as signal
import numpy as np
from matplotlib import pyplot as plt
import time
from torch2trt import TRTModule
from torch2trt import torch2trt
from models.CNN import ErnNet
from utils import get_pytorch_model

In [3]:
def get_start_points_x(width, slice_width, overlap_x):
    x_points = [0]
    stride = int(slice_width * (1 - overlap_x))
    counter = 1
    while True:
        pt = stride * counter
        if pt + slice_width >= width:
            x_points.append(width - slice_width)
            break
        else:
            x_points.append(pt)
        counter += 1
    return x_points

def get_total_inference_time(self, model, num_loops):
    """Returns the total inference time on all the loops"""
    # Wait for all kernels in all streams on the CUDA device to complete.
    torch.cuda.current_stream().synchronize()

    # GPU warmup
    for _ in range(10):
        _ = model(self.input_data_batch)

    t0 = time.time()
    for _ in range(num_loops):
        _ = model(self.input_data_batch)
        torch.cuda.current_stream().synchronize()
    t1 = time.time()

    return t1 - t0

In [2]:
# Load model
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('trt_models/model_trt_int8.pth'))

<All keys matched successfully>

In [84]:
fs = 714 # Sampling frequency
nperseg = 128 # Length of each segment
noverlap = 64 # Number of overlapping points between segments
nfft = 128 # Length of FFT
dwell_time = 4
num_points = np.int64(np.floor(dwell_time* fs))
print(num_points)

# Remember what Robert said about 64 and 32 when creating arrays

x = np.random.random(num_points) + np.random.random(num_points) * 1j

f, t, Zxx = signal.stft(x, fs=fs, window='hamming', nperseg=nperseg, noverlap=noverlap, nfft=nfft, detrend=False, return_onesided=True, boundary='zeros', padded=False, axis=- 1)
# plt.pcolormesh(t, f, np.abs(Zxx))

fft_dB =20*np.log10(np.abs(Zxx))

fft_dB.shape

2856


(128, 45)

In [81]:
714*45

32130

In [49]:
x = torch.randn(3,2, dtype=torch.cfloat)
x

tensor([[-0.6035-0.7965j, -1.0970+0.3573j],
        [-0.0987+0.5487j,  0.1597+1.0346j],
        [ 0.4414+0.4647j,  0.8101-0.7251j]])

In [9]:
x = torch.randn(40000,num_points, dtype=torch.cfloat)
print(len(x))

# Get the short-term fourier transform of the signal
f, t, Zxx = signal.stft(x, fs=fs, window='hamming', nperseg=nperseg, noverlap=noverlap, nfft=nfft, detrend=False, return_onesided=True, boundary='zeros', padded=False, axis=- 1)
# plt.pcolormesh(t, f, np.abs(Zxx))

Zxx_tensor = torch.from_numpy(Zxx)

# print(type(Zxx))
# # Converting to dB
# Zxx_tensor = torch.from_numpy(Zxx)
# fft_dB = 20*torch.log10(torch.abs(Zxx_tensor))
# # fft_dB.shape

# width = fft_dB.shape[1]
# fft_dB.shape


Zxx_tensor = torch.from_numpy(Zxx)

# Zxx = torch.randn(128,45)

# Converting to dB
fft_dB = 20*torch.log10(torch.abs(Zxx_tensor))
_ = model_trt(fft_dB.cuda())

40000


In [59]:
num_points

2856

In [6]:
ran
fs = 714 # Sampling frequency
nperseg = 128 # Length of each segment
noverlap = 64 # Number of overlapping points between segments
nfft = 128 # Length of FFT
dwell_time = 4
slice_width = 45
overlap = 0.5
num_points = np.int64(np.floor(dwell_time* fs))
classified_imgs = 0


torch.cuda.current_stream().synchronize()
# GPU warmup
for _ in range(100):
    _ = model_trt(torch.randn((1, 1, 128, 45)).cuda())

# Start counter
t0 = time.time()
t1 = time.time()
while (t1 - t0) < dwell_time:

    # Creating the samples
    x = torch.randn(num_points, dtype=torch.cfloat)
    # Get the short-term fourier transform of the signal
    f, t, Zxx = signal.stft(x, fs=fs, window='hamming', nperseg=nperseg, noverlap=noverlap, nfft=nfft, detrend=False, return_onesided=True, boundary='zeros', padded=False, axis=-1)
    Zxx_tensor = torch.from_numpy(Zxx)

    # Zxx = torch.randn(128,45)

    # Converting to dB
    fft_dB = 20*torch.log10(torch.abs(Zxx_tensor))
    _ = model_trt(fft_dB.cuda())

    classified_imgs += 1

    t1 = time.time()

print(classified_imgs-1)

/home/nyasha/anaconda3/envs/thesis/lib/python3.8/site-packages/scipy/signal/spectral.py:1812: UserWarning: Input data is complex, switching to return_onesided=False
  warnings.warn('Input data is complex, switching to '


8986


In [12]:
# On cpu
# On gpu
# On cpu with parallelisation

# Show the throttling of the stft function
# Review decision in padded=false
#
num_imgs

5500

In [39]:
# In parallel

33000

In [21]:
throughput = num_imgs/(t1-t0)
point_throughput = throughput * slice_width
supported_range_bins = point_throughput/fs
supported_range_bins*5

1969.864781229934